# 通过重定向/管道/文件接受输入

```python
#!/usr/bin/env python3
import fileinput

with fileinput.input() as f_input:
    for line in f_input:
        print(line, end='')
```

```bash
$ ls | ./filein.py          # Prints a directory listing to stdout.
$ ./filein.py /etc/passwd   # Reads /etc/passwd to stdout.
$ ./filein.py < /etc/passwd # Reads /etc/passwd to stdout.
```

# 终止程序并给出错误信息

```python
raise SystemExit('It failed!')
```

将消息在 `sys.stderr` 中打印，然后程序以状态码`1`退出

# 解析命令行选项

```python
# search.py
'''
Hypothetical command-line tool for searching a collection of
files for one or more text patterns.
'''
import argparse
parser = argparse.ArgumentParser(description='Search some files')

parser.add_argument(dest='filenames',metavar='filename', nargs='*')

# 允许某个参数重复出现多次，并将它们追加到一个列表中去。 required 标志表示该参数至少要有一个。-p 和 --pat 表示两个参数名形式都可使用。
parser.add_argument('-p', '--pat',metavar='pattern', required=True,
                    dest='patterns', action='append',
                    help='text pattern to search for')

# 根据参数是否存在来设置一个 Boolean 标志
parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')

# 接受一个单独值并将其存储为一个字符串
parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')

# 接受一个值，但是会将其和可能的选择值做比较，以检测其合法性
parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')

args = parser.parse_args()

# Output the collected arguments
print(args.filenames)
print(args.patterns)
print(args.verbose)
print(args.outfile)
print(args.speed)
```

```bash
bash % python3 search.py -h
usage: search.py [-h] [-p pattern] [-v] [-o OUTFILE] [--speed {slow,fast}]
                 [filename [filename ...]]

Search some files

positional arguments:
  filename

optional arguments:
  -h, --help            show this help message and exit
  -p pattern, --pat pattern
                        text pattern to search for
  -v                    verbose mode
  -o OUTFILE            output file
  --speed {slow,fast}   search speed
```

# 运行时弹出密码输入提示

In [8]:
import getpass

user = getpass.getuser()
passwd = getpass.getpass()

def svc_login(user,passwd):
    print(user,passwd)
    return True

if svc_login(user, passwd):    # You must write svc_login()
   print('Yay!')
else:
   print('Boo!')

 ··········


huzhenyu test!@1234
Yay!


`getpass.getuser()` 不会弹出用户名的输入提示。 它会根据该用户的shell环境或者会依据本地系统的密码库（支持 pwd 模块的平台）来使用当前用户的登录名

# 获取终端的大小

```python
>>> import os
>>> sz = os.get_terminal_size()
>>> sz
os.terminal_size(columns=138, lines=16)
>>> sz.columns
138
>>> sz.lines
16
```

# 执行外部命令并获取它的输出

In [11]:
import subprocess
out_bytes = subprocess.check_output(['ls','-a'])

In [15]:
print(out_bytes.decode('utf-8'))

.
..
.ipynb_checkpoints
Metaprogramming.ipynb
class_object.ipynb
dataStructure_algorithm.ipynb
express_evaluator.py
file_io.ipynb
function.ipynb
module_package.ipynb
multitask.ipynb
network_web.ipynb
number&date&time.ipynb
script_system.ipynb
str&text.ipynb
数据编码和处理.ipynb



捕获到错误并获取返回
```python
try:
    out_bytes = subprocess.check_output(['cmd','arg1','arg2'])
except subprocess.CalledProcessError as e:
    out_bytes = e.output       # Output generated before error
    code      = e.returncode   # Return code
```

默认情况下，check_output() 仅仅返回输入到标准输出的值。 如果你需要同时收集标准输出和错误输出，使用 `stderr` 参数:
```python
out_bytes = subprocess.check_output(['cmd','arg1','arg2'],
                                stderr=subprocess.STDOUT)
```

超时机制来执行命令，使用 `timeout` 参数：
```python
try:
    out_bytes = subprocess.check_output(['cmd','arg1','arg2'], timeout=5)
except subprocess.TimeoutExpired as e:
    ...
```

让命令被一个shell执行，传递一个字符串参数，并设置参数 `shell=True`
```python
out_bytes = subprocess.check_output('grep python | wc > out', shell=True)
```

In [21]:
import subprocess

# Some text to send
text = b'''
hello world
this is a test
goodbye
'''

# Launch a command with pipes
p = subprocess.Popen(['wc'],
          stdout = subprocess.PIPE,
          stdin = subprocess.PIPE)

# Send the data and get the output
stdout, stderr = p.communicate(text)

# To interpret as text, decode
out = stdout.decode('utf-8')
if stderr is not None:
    err = stderr.decode('utf-8')

In [19]:
out

'       4       7      36\n'

# 复制或者移动文件和目录

```python
import shutil

# Copy src to dst. (cp src dst)
shutil.copy(src, dst)

# Copy files, but preserve metadata (cp -p src dst)
shutil.copy2(src, dst)

# Copy directory tree (cp -R src dst)
shutil.copytree(src, dst)

# Move src to dst (mv src dst)
shutil.move(src, dst)
```

默认情况下，对于符号链接而已这些命令处理的是它指向的东西。 例如，如果源文件是一个符号链接，那么目标文件将会是符号链接指向的文件。 如果你只想复制符号链接本身，那么需要指定关键字参数 `follow_symlinks`
```python
shutil.copytree(src, dst, symlinks=True)
```

`copytree()` 可以让你在复制过程中选择性的忽略某些文件或目录。 你可以提供一个忽略函数，接受一个目录名和文件名列表作为输入，返回一个忽略的名称列表。
```python
def ignore_pyc_files(dirname, filenames):
    return [name in filenames if name.endswith('.pyc')]

shutil.copytree(src, dst, ignore=ignore_pyc_files)
```

忽略某种模式的文件名
```python
shutil.copytree(src, dst, ignore=shutil.ignore_patterns('*~', '*.pyc'))
```

在复制过程中，函数可能会碰到损坏的符号链接，因为权限无法访问文件的问题等等。 为了解决这个问题，所有碰到的问题会被收集到一个列表中并打包为一个单独的异常，到了最后再抛出。
```python
try:
    shutil.copytree(src, dst)
except shutil.Error as e:
    for src, dst, msg in e.args[0]:
         # src is source name
         # dst is destination name
         # msg is error message from exception
         print(dst, src, msg)
```

关键字参数 `ignore_dangling_symlinks=True` ， 这时候 copytree() 会忽略掉无效符号链接。

# 创建和解压归档文件

```python
>>> import shutil
>>> shutil.unpack_archive('Python-3.3.0.tgz')

>>> shutil.make_archive('py33','zip','Python-3.3.0')
'/Users/beazley/Downloads/py33.zip'
>>> shutil.get_archive_formats()
[('bztar', "bzip2'ed tar-file"), ('gztar', "gzip'ed tar-file"),
 ('tar', 'uncompressed tar file'), ('zip', 'ZIP file')]
```

# 通过文件名查找文件

In [23]:
#!/usr/bin/env python3.3
import os

def findfile(start, name):
    for relpath, dirs, files in os.walk(start):
        if name in files:
            full_path = os.path.join(start, relpath, name)
            print(os.path.normpath(os.path.abspath(full_path)))

def main():
    findfile(sys.argv[1], sys.argv[2])

In [24]:
findfile('.','script_system.ipynb')

/Users/huzhenyu/Documents/personal/python_personal/python/cookbook/script_system.ipynb


`os.walk()` 方法为我们遍历目录树， 每次进入一个目录，它会返回一个三元组，包含相对于查找目录的相对路径，一个该目录下的目录名列表， 以及那个目录下面的文件名列表。  
`os.path.join()` 合并路径。  
`os.path.abspath()` ,它接受一个路径，可能是相对路径，最后返回绝对路径。  
`os.path.normpath()` ，用来返回正常路径，可以解决双斜杆、对目录的多重引用的问题等。  

In [25]:
import os
import time

def modified_within(top, seconds):
    now = time.time()
    for path, dirs, files in os.walk(top):
        for name in files:
            fullpath = os.path.join(path, name)
            if os.path.exists(fullpath):
                mtime = os.path.getmtime(fullpath)
                if mtime > (now - seconds):
                    print(fullpath)
def main():
    import sys
    if len(sys.argv) != 3:
        print('Usage: {} dir seconds'.format(sys.argv[0]))
        raise SystemExit(1)

    modified_within(sys.argv[1], float(sys.argv[2]))

In [26]:
modified_within('.',1000)

./script_system.ipynb


# 读取配置文件

In [27]:
ini = """
; config.ini
; Sample configuration file

[installation]
library=%(prefix)s/lib
include=%(prefix)s/include
bin=%(prefix)s/bin
prefix=/usr/local

# Setting related to debug configuration
[debug]
log_errors=true
show_warnings=False

[server]
port: 8080
nworkers: 32
pid-file=/tmp/spam.pid
root=/www/root
signature:
    =================================
    Brought to you by the Python Cookbook
    =================================
"""

In [28]:
from io import StringIO
f = StringIO(ini)

In [36]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read_string(ini)

In [38]:
cfg.sections(),cfg.get('installation','library')

(['installation', 'debug', 'server'], '/usr/local/lib')

In [39]:
cfg.set('server','port','9000')

In [41]:
cfg.set('debug','log_errors','False')

In [42]:
cfg.getint('server','port')

9000

In [43]:
import sys
cfg.write(sys.stdout)

[installation]
library = %(prefix)s/lib
include = %(prefix)s/include
bin = %(prefix)s/bin
prefix = /usr/local

[debug]
log_errors = False
show_warnings = False

[server]
port = 9000
nworkers = 32
pid-file = /tmp/spam.pid
root = /www/root
signature = 
	Brought to you by the Python Cookbook



配置文件的语法要更自由些，下面的赋值语句是等效的：
```ini
prefix=/usr/local
prefix: /usr/local
```

配置文件中的名字是不区分大小写的。  
在解析值的时候，`getboolean()` 方法查找任何可行的值。
```python
log_errors = true
log_errors = TRUE
log_errors = Yes
log_errors = 1
```

文件是安装一个整体被读取的。如果碰到了变量替换，它实际上已经被替换完成了。  
变量在使用它的变量之前或之后定义都是可以的。  
变量的改写采取的是后发制人策略，以最后一个为准。

# 给简单脚本增加日志功能

In [52]:
import logging

def main():
    # Configure the logging system
    logging.basicConfig(
        filename='app.log',
        level=logging.ERROR
    )

    # Variables (to make the calls that follow work)
    hostname = 'www.python.org'
    item = 'spam'
    filename = 'data.csv'
    mode = 'r'

    # Example logging calls (insert into your program)
    logging.critical('Host %s unknown', hostname)
    logging.error("Couldn't find %r", item)
    logging.warning('Feature is deprecated')
    logging.info('Opening file %r, mode=%r', filename, mode)
    logging.debug('Got here')

if __name__ == '__main__':
    main()

In [53]:
logging.basicConfig(
     filename='app.log',
     level=logging.WARNING,
     format='%(levelname)s:%(asctime)s:%(message)s')

```python
import logging
import logging.config

def main():
    # Configure the logging system
    logging.config.fileConfig('logconfig.ini')
    ...
```

+ logconfig.ini  

```ini
[loggers]
keys=root

[handlers]
keys=defaultHandler

[formatters]
keys=defaultFormatter

[logger_root]
level=INFO
handlers=defaultHandler
qualname=root

[handler_defaultHandler]
class=FileHandler
formatter=defaultFormatter
args=('app.log', 'a')

[formatter_defaultFormatter]
format=%(levelname)s:%(name)s:%(message)s
```

In [55]:
logging.basicConfig(level=logging.INFO)

In [57]:
# basicConfig() 在程序中只能被执行一次。如果你稍后想改变日志配置， 就需要先获取 root logger ，然后直接修改它。
logging.getLogger().level = logging.DEBUG

# 给函数库增加日志功能

In [58]:
# somelib.py

import logging
log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())

# Example function (for testing)
def func():
    log.critical('A Critical Error!')
    log.debug('A debug message')

默认情况下不会打印日志
```python
import somelib
somelib.func()
```

如果配置过日志系统，那么日志消息打印就开始生效
```python
import logging
logging.basicConfig()
somelib.func()
>>> CRITICAL:somelib:A Critical Error!
```

# 实现一个计时器

In [62]:
import time

class Timer:
    def __init__(self, func=time.perf_counter):
        self.elapsed = 0.0
        self._func = func
        self._start = None

    def start(self):
        if self._start is not None:
            raise RuntimeError('Already started')
        self._start = self._func()

    def stop(self):
        if self._start is None:
            raise RuntimeError('Not started')
        end = self._func()
        self.elapsed += end - self._start
        self._start = None

    def reset(self):
        self.elapsed = 0.0

    @property
    def running(self):
        return self._start is not None

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

In [63]:
def countdown(n):
    while n > 0:
        n -= 1

# Use 1: Explicit start/stop
t = Timer()
t.start()
countdown(1000000)
t.stop()
print(t.elapsed)

# Use 2: As a context manager
with t:
    countdown(1000000)

print(t.elapsed)

with Timer() as t2:
    countdown(1000000)
print(t2.elapsed)

0.07213246100582182
0.12283272201602813
0.0449462269898504


`time.perf_counter()` 函数可以确保使用系统上面最精确的计时器。   
`time.process_time()` 计算该进程所花费的CPU时间。

# 限制内存和CPU的使用量

In [67]:
# 限制cpu
import signal
import resource
import os

def time_exceeded(signo, frame):
    print("Time's up!")
    raise SystemExit(1)

def set_max_runtime(seconds):
    # Install the signal handler and set a resource limit
    soft, hard = resource.getrlimit(resource.RLIMIT_CPU)
    resource.setrlimit(resource.RLIMIT_CPU, (seconds, hard))
    signal.signal(signal.SIGXCPU, time_exceeded)

def main():
    set_max_runtime(15)
    while True:
        pass

In [68]:
# 限制内存
import resource

def limit_memory(maxsize):
    soft, hard = resource.getrlimit(resource.RLIMIT_AS)
    resource.setrlimit(resource.RLIMIT_AS, (maxsize, hard))

In [69]:
soft, hard = resource.getrlimit(resource.RLIMIT_CPU)

In [70]:
soft,hard

(9223372036854775807, 9223372036854775807)

In [71]:
soft, hard = resource.getrlimit(resource.RLIMIT_AS)

In [72]:
soft,hard

(9223372036854775807, 9223372036854775807)

# 启动一个WEB浏览器

In [73]:
import webbrowser
webbrowser.open('http://www.python.org')

True

In [74]:
webbrowser.open_new('http://www.python.org')

True

In [75]:
webbrowser.open_new_tab('http://www.python.org')

True